# 🎯 A/B Тестирование: Основы и Практика

## Студенческая презентация (20 минут)

---

**Содержание:**
1. Что такое A/B тест и зачем он нужен?
2. Ошибки I и II рода — главные враги аналитика
3. Как понять, сколько нужно пользователей? (MDE)
4. Проверка валидности критерия (AA-тесты)
5. Агрегация данных — почему нельзя использовать сырые логи
6. Метрики отношения и дельта-метод
7. Bootstrap — универсальный инструмент
8. Методы повышения чувствительности (CUPED, стратификация)
9. Проблема подглядывания (peeking problem)
10. Множественное тестирование

---

In [ ]:
import numpy as np
import scipy.stats as stats
from tqdm import tqdm
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

---

# 1. 🔬 Что такое A/B тест?

## Как менялся Яндекс.Поиск со временем

<div style="display: flex;">
    <img src="pictures/Яндекс2000.jpg" width="250">
    <img src="pictures/Яндекс2007_2009.jpg" width="250">
    <img src="pictures/Яндекс2016_2017.jpg" width="250">
</div>

### Современная версия
<img src="pictures/Яндекс2025.png" width="500">

**В зрелых продуктах любое изменение — это тысячи проведённых экспериментов!**

## Практический кейс: Яндекс.Такси

**Тестируем новый функционал:** В высокий спрос пишем пользователю, что если подождать X минут — цена упадёт на Y рублей.

<img src="pictures/trips.png" width="500">

### Что могло произойти без A/B теста?
- 🌧️ Ухудшилась погода
- 🎄 Предновогодняя неделя
- ✅ Действительно крутую фичу предложили

**Без эксперимента мы не знаем, что именно повлияло на метрики!**

## Определение A/B тестирования

> **A/B-тестирование** — способ определить влияние изменений на продукт, **исключив внешние факторы**.

### Как это работает:
1. Разбиваем пользователей на **контрольную (A)** и **тестовую (B)** группы
2. Контрольная группа — без изменений
3. Тестовая группа — включаем новую фичу
4. Сравниваем результаты

### Ограничения:
- ⏱️ Требуется время на проведение
- 👥 Нужна большая клиентская база
- 🏗️ Требуется развитая инфраструктура

### Платформа экспериментов Laba
<img src="pictures/laba.png" width="700">

## Типы метрик в A/B тестах

<img src="pictures/metrics.png" width="700">

### Value метрики (абсолютные):
- **GMV** — общая стоимость поездок
- **Trips** — количество заказов
- **Supply Hours** — часы работы водителей

### Ratio метрики (отношения):
- **Acceptance Rate** = принятые офферы / показанные офферы
- **Completed Rate** = завершённые поездки / назначенные заказы
- **Конверсии** — пришёл → увидел → купил

---

# 2. ⚠️ Ошибки I и II рода

## Два типа ошибок в статистике

<img src="pictures/errors.png" width="600">

### Ошибка I рода (α) — False Positive
> Отклонили H₀, когда она верна  
> **"Нашли эффект, которого нет"**

### Ошибка II рода (β) — False Negative  
> Не отклонили H₀, когда верна H₁  
> **"Не нашли эффект, который есть"**

### Мощность теста
> **Мощность = 1 - β** — вероятность найти эффект, если он есть

## Связь между ошибками

<img src="pictures/errors2.png" width="700">

### Ключевые зависимости:
- α ↓ ⟹ β ↑ (строже порог → чаще пропускаем эффект)
- α ↑ ⟹ β ↓ (мягче порог → чаще ложные срабатывания)
- Эффект ↑ ⟹ β ↓ (большой эффект легче найти)

### Индустриальный стандарт:
- **α = 0.05** (5% ложных срабатываний)
- **β = 0.20** (80% мощность)

In [ ]:
# Визуализация ошибок I и II рода
fig, ax = plt.subplots(figsize=(12, 6))

x = np.linspace(-4, 8, 1000)

# H0: эффекта нет (среднее = 0)
y_h0 = stats.norm.pdf(x, 0, 1)

# H1: эффект есть (среднее = 3)
effect = 3
y_h1 = stats.norm.pdf(x, effect, 1)

# Критическое значение для α = 0.05 (односторонний)
alpha = 0.05
critical_value = stats.norm.ppf(1 - alpha)

ax.plot(x, y_h0, 'b-', lw=2, label='H₀: Эффекта нет')
ax.plot(x, y_h1, 'r-', lw=2, label=f'H₁: Эффект = {effect}')

# Закрашиваем область ошибки I рода (α)
x_alpha = x[x >= critical_value]
ax.fill_between(x_alpha, 0, stats.norm.pdf(x_alpha, 0, 1), 
                color='blue', alpha=0.3, label=f'α (Ошибка I рода) = {alpha}')

# Закрашиваем область ошибки II рода (β)
x_beta = x[x <= critical_value]
beta = stats.norm.cdf(critical_value, effect, 1)
ax.fill_between(x_beta, 0, stats.norm.pdf(x_beta, effect, 1), 
                color='red', alpha=0.3, label=f'β (Ошибка II рода) = {beta:.2f}')

ax.axvline(critical_value, color='green', linestyle='--', lw=2, 
           label=f'Критическое значение = {critical_value:.2f}')

ax.set_xlabel('Значение статистики', fontsize=12)
ax.set_ylabel('Плотность вероятности', fontsize=12)
ax.set_title('Ошибки I и II рода в статистическом тестировании', fontsize=14)
ax.legend(loc='upper right')
ax.set_ylim(0, 0.45)
plt.tight_layout()
plt.show()

print(f"Мощность теста (1-β) = {1-beta:.2f}")

---

# 3. 📊 MDE — Минимальный детектируемый эффект

## Что такое MDE?

> **MDE (Minimal Detectable Effect)** — минимальная разница между группами, которую тест может обнаружить с заданной вероятностью.

<img src="pictures/power.png" width="600">

### Зачем нужен MDE?
- 📏 Планирование размера выборки
- 💰 Оценка стоимости эксперимента
- 🎯 Понимание чувствительности теста

## Формула для расчёта размера выборки

$$m > \dfrac{(z_{1-\alpha/2} + z_{1-\beta})^2 (1 + k)\sigma^2}{e^2}$$

Где:
- $m$ — размер контрольной группы
- $k = m/n$ — отношение размеров групп
- $\sigma^2$ — дисперсия метрики
- $e$ — ожидаемый эффект
- $z_{1-\alpha/2}$ ≈ 1.96 для α = 0.05
- $z_{1-\beta}$ ≈ 0.84 для β = 0.20

In [ ]:
def calculate_sample_size(sigma, effect, k=1, alpha=0.05, beta=0.2):
    """Расчёт необходимого размера выборки"""
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(1 - beta)
    const = (z_alpha + z_beta)**2
    m = const * (1 + k) * sigma**2 / (effect**2)
    return int(m)

def calculate_mde(sigma, num_users, k=1, alpha=0.05, beta=0.2):
    """Расчёт MDE для заданного размера выборки"""
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(1 - beta)
    m = num_users * k / (k + 1)
    return (z_alpha + z_beta) * sigma * np.sqrt((1 + k) / m)

# Пример: денежная метрика со средним 500 рублей
mu = 500
sigma = 500  # Для экспоненциального распределения σ ≈ μ

# Сколько нужно пользователей для эффекта 2%?
effect = mu * 0.02  # 2% от среднего
sample_size = calculate_sample_size(sigma, effect, k=1)
print(f"Для детектирования 2% эффекта нужно {sample_size:,} пользователей в каждой группе")
print(f"Всего: {sample_size * 2:,} пользователей")

# Какой MDE можем детектировать при 100К пользователей?
num_users = 100000
mde = calculate_mde(sigma, num_users, k=1)
print(f"\nПри {num_users:,} пользователей MDE = {mde:.1f} руб. ({100*mde/mu:.1f}%)")

In [ ]:
# Визуализация: MDE vs размер выборки
sample_sizes = np.logspace(3, 6, 50)  # от 1K до 1M
mdes = [calculate_mde(sigma, n) for n in sample_sizes]
mdes_percent = [100 * m / mu for m in mdes]

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(sample_sizes, mdes_percent, 'b-', lw=2)
ax.axhline(5, color='red', linestyle='--', label='5% эффект')
ax.axhline(2, color='green', linestyle='--', label='2% эффект')
ax.axhline(1, color='orange', linestyle='--', label='1% эффект')

ax.set_xscale('log')
ax.set_xlabel('Размер выборки (всего пользователей)', fontsize=12)
ax.set_ylabel('MDE (%)', fontsize=12)
ax.set_title('Зависимость MDE от размера выборки', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

# 4. ✅ Проверка валидности критерия (AA-тесты)

## Что такое синтетический AA-тест?

> **AA-тест** — эксперимент без эффекта для проверки работоспособности критерия.

### Что проверяем:
- При H₀ (нет эффекта) p-value должно быть **равномерно распределено** на [0, 1]
- Ошибка I рода должна быть ≈ α (обычно 5%)

<img src="pictures/crit.png" width="500">

In [ ]:
# Функция для проверки валидности критерия
def plot_pvalue_validation(pvalue_vector, title, color='skyblue'):
    """Визуализация валидности статистического критерия"""
    pvalue_vector = np.array(pvalue_vector)
    n = len(pvalue_vector)
    
    # Расчёт ошибки I рода и доверительного интервала
    alpha_hat = np.mean(pvalue_vector < 0.05)
    ci_width = 1.96 * np.sqrt(alpha_hat * (1 - alpha_hat) / n)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 1. Гистограмма p-value
    axes[0].hist(pvalue_vector, bins=20, edgecolor='black', alpha=0.7, color=color, density=True)
    axes[0].axhline(1, color='red', linestyle='--', label='Равномерное')
    axes[0].set_xlabel('p-value')
    axes[0].set_ylabel('Плотность')
    axes[0].set_title(f'Гистограмма p-value\nОшибка I рода: {alpha_hat:.3f}')
    axes[0].legend()
    
    # 2. QQ-plot
    stats.probplot(pvalue_vector, dist=stats.uniform, plot=axes[1])
    axes[1].get_lines()[1].set_linestyle('--')
    axes[1].set_title('QQ-plot vs Uniform[0,1]')
    
    # 3. Эмпирическая функция распределения
    pvals_sorted = np.sort(pvalue_vector)
    ecdf = np.arange(1, n+1) / n
    axes[2].step(pvals_sorted, ecdf, where='post', label='Эмпирическая ФР', color=color)
    axes[2].plot([0, 1], [0, 1], 'r--', label='Теоретическая (Uniform)')
    axes[2].set_xlabel('p-value')
    axes[2].set_ylabel('F(p-value)')
    axes[2].set_title('Эмпирическая vs Теоретическая ФР')
    axes[2].legend()
    
    plt.suptitle(f'{title}', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Тест Колмогорова-Смирнова
    ks_stat, ks_pval = stats.kstest(pvalue_vector, 'uniform')
    print(f"Ошибка I рода: {alpha_hat:.4f} [{alpha_hat-ci_width:.4f}, {alpha_hat+ci_width:.4f}]")
    print(f"KS-тест: статистика = {ks_stat:.4f}, p-value = {ks_pval:.4f}")
    if ks_pval > 0.05:
        print("✅ Критерий валиден (p-value равномерно)")
    else:
        print("❌ Критерий НЕ валиден!")

In [ ]:
# Синтетический AA-тест для t-теста
np.random.seed(42)
pvalues_aa = []

for _ in range(1000):
    # Генерируем две группы без эффекта
    control = np.random.exponential(500, 1000)
    test = np.random.exponential(500, 1000)  # Тот же параметр!
    
    _, pval = stats.ttest_ind(test, control, equal_var=False)
    pvalues_aa.append(pval)

plot_pvalue_validation(pvalues_aa, 't-test на экспоненциальных данных (AA-тест)')

---

# 5. 📦 Агрегация данных

## Почему нельзя использовать сырые логи?

<img src="pictures/raw_logs.png" width="400">

### Проблема зависимости данных

Один пользователь генерирует **много событий** → события **зависимы** → критерий работает неправильно!

### Решение: Поюзерная агрегация

<img src="pictures/sum.png" width="300">

> **Один пользователь = одно наблюдение**

In [ ]:
# Демонстрация проблемы зависимых данных
def generate_user_events(n_users=1000, mu_exp=500, mu_pois=3):
    """Генерация данных: пользователи с несколькими событиями"""
    noise_factors = [0.8, 0.9, 1, 1.1, 1.2]
    events = []  # Сырые события
    users = []   # Агрегированные по пользователям
    
    for _ in range(n_users):
        # Количество событий у пользователя (Пуассон)
        n_events = np.random.poisson(mu_pois)
        if n_events == 0:
            n_events = 1
        
        # Базовое значение для пользователя
        base_value = np.random.exponential(mu_exp)
        
        # События с шумом
        user_events = [base_value * random.choice(noise_factors) for _ in range(n_events)]
        events.extend(user_events)
        users.append(sum(user_events))
    
    return events, users

# Сравним критерий на сырых данных vs агрегированных
np.random.seed(42)
pvalues_raw = []
pvalues_agg = []

for _ in range(1000):
    test_events, test_users = generate_user_events()
    ctrl_events, ctrl_users = generate_user_events()
    
    _, pval_raw = stats.ttest_ind(test_events, ctrl_events, equal_var=False)
    _, pval_agg = stats.ttest_ind(test_users, ctrl_users, equal_var=False)
    
    pvalues_raw.append(pval_raw)
    pvalues_agg.append(pval_agg)

In [ ]:
# Сырые данные — критерий НЕ работает!
plot_pvalue_validation(pvalues_raw, '❌ Сырые события (зависимые данные)', 'salmon')

In [ ]:
# Агрегированные данные — критерий работает
plot_pvalue_validation(pvalues_agg, '✅ Поюзерная агрегация (независимые данные)', 'lightgreen')

## Виды агрегации

<img src="pictures/agg.png" width="300">

### 1. Поюзерная агрегация
- Один пользователь = одно наблюдение
- ✅ Независимость данных
- ✅ Понятная интерпретация

### 2. Побакетная агрегация
- Группируем пользователей в "бакеты"
- ✅ Снижает влияние выбросов
- ❌ Меньше наблюдений → меньше мощность

---

# 6. 📐 Метрики отношения и Дельта-метод

## Проблема с ratio-метриками

Хотим сравнить **глобальное отношение**:

$$\text{Ratio} = \dfrac{\sum X}{\sum Y} = \dfrac{\bar{X}}{\bar{Y}}$$

### Примеры:
- **Средний чек** = GMV / Trips
- **Конверсия** = Клики / Показы
- **ARPU** = Revenue / Users

**Проблема:** t-test работает с одним вектором, а у нас два (числитель и знаменатель)!

## Дельта-метод

<img src="pictures/delta_ratio_new.png" width="700">

### Формула дисперсии для отношения:

$$\text{Var}\left(\frac{\bar{X}}{\bar{Y}}\right) \approx \dfrac{1}{n} \cdot \dfrac{1}{\bar{Y}^2} \left[ \hat\sigma_X^2 - 2\dfrac{\bar{X}}{\bar{Y}}\hat\sigma_{XY} + \dfrac{\bar{X}^2}{\bar{Y}^2} \hat\sigma_Y^2 \right]$$

In [ ]:
def delta_variance_ratio(numerator, denominator):
    """Дисперсия отношения дельта-методом"""
    x = np.array(numerator)
    y = np.array(denominator)
    n = len(x)
    
    mu_x = np.mean(x)
    mu_y = np.mean(y)
    var_x = np.var(x, ddof=1)
    var_y = np.var(y, ddof=1)
    cov_xy = np.cov(x, y, ddof=1)[0, 1]
    
    # Формула дельта-метода
    delta_var = (1/n) * (1/mu_y**2) * (var_x - 2*(mu_x/mu_y)*cov_xy + (mu_x**2/mu_y**2)*var_y)
    return delta_var

def delta_test_ratio(x_num, x_denom, y_num, y_denom):
    """t-тест для ratio метрик с дельта-методом"""
    n, m = len(x_num), len(y_num)
    
    # Оценки отношений
    ratio_test = np.mean(x_num) / np.mean(x_denom)
    ratio_ctrl = np.mean(y_num) / np.mean(y_denom)
    
    # Дисперсии
    var_test = delta_variance_ratio(x_num, x_denom)
    var_ctrl = delta_variance_ratio(y_num, y_denom)
    
    # Статистика и p-value
    delta = ratio_test - ratio_ctrl
    se = np.sqrt(var_test + var_ctrl)
    t_stat = delta / se
    p_value = 2 * stats.t.sf(np.abs(t_stat), n + m - 2)
    
    return p_value

# Генерация данных для ratio-метрики
def generate_ratio_data(n_users=10000, events_per_user=5, effect=0):
    """Генерация данных для метрики отношения (например, средний чек)"""
    mu = np.random.uniform(300, 1000, size=n_users)
    n_events = np.random.poisson(events_per_user - 1, size=n_users) + 1
    
    numerator = []
    denominator = []
    
    for i in range(n_users):
        user_events = np.random.exponential(mu[i], n_events[i])
        numerator.append((1 + effect) * np.sum(user_events))
        denominator.append(n_events[i])
    
    return numerator, denominator

In [ ]:
# Проверка валидности дельта-метода
np.random.seed(42)
pvalues_delta = []

for _ in range(1000):
    x_num, x_denom = generate_ratio_data(n_users=5000)
    y_num, y_denom = generate_ratio_data(n_users=5000)
    
    pval = delta_test_ratio(x_num, x_denom, y_num, y_denom)
    pvalues_delta.append(pval)

plot_pvalue_validation(pvalues_delta, 'Дельта-метод для ratio метрики (AA-тест)', 'lightblue')

---

# 7. 🎰 Bootstrap — универсальный инструмент

## Что такое Bootstrap?

> **Bootstrap** — метод оценки характеристик распределения через повторную выборку с возвращением.

### Алгоритм:
1. Из выборки размера n делаем B выборок размера n **с возвращением**
2. Для каждой считаем интересующую статистику
3. Получаем распределение статистики

### Преимущества:
- ✅ Не требует предположений о распределении
- ✅ Работает для любых статистик
- ✅ Автоматически учитывает зависимости

In [ ]:
def bootstrap_test(test_data, control_data, n_bootstrap=1000, alpha=0.05):
    """
    Bootstrap-тест для сравнения средних.
    Возвращает p-value и доверительный интервал для разницы.
    """
    observed_diff = np.mean(test_data) - np.mean(control_data)
    
    # Bootstrap выборки
    bootstrap_diffs = []
    for _ in range(n_bootstrap):
        boot_test = np.random.choice(test_data, size=len(test_data), replace=True)
        boot_ctrl = np.random.choice(control_data, size=len(control_data), replace=True)
        bootstrap_diffs.append(np.mean(boot_test) - np.mean(boot_ctrl))
    
    bootstrap_diffs = np.array(bootstrap_diffs)
    
    # p-value (двусторонний)
    # Центрируем относительно 0 под H0
    centered_diffs = bootstrap_diffs - observed_diff
    p_value = 2 * min(
        np.mean(centered_diffs >= observed_diff),
        np.mean(centered_diffs <= observed_diff)
    )
    
    # Доверительный интервал
    ci_lower = np.percentile(bootstrap_diffs, 100 * alpha / 2)
    ci_upper = np.percentile(bootstrap_diffs, 100 * (1 - alpha / 2))
    
    return p_value, (ci_lower, ci_upper), bootstrap_diffs

# Пример использования
np.random.seed(42)
control = np.random.exponential(500, 1000)
test = np.random.exponential(520, 1000)  # Эффект 4%

pval, ci, boot_diffs = bootstrap_test(test, control, n_bootstrap=2000)

print(f"Наблюдаемая разница: {np.mean(test) - np.mean(control):.2f}")
print(f"Bootstrap p-value: {pval:.4f}")
print(f"95% ДИ: [{ci[0]:.2f}, {ci[1]:.2f}]")

In [ ]:
# Визуализация bootstrap распределения
fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(boot_diffs, bins=50, edgecolor='black', alpha=0.7, color='steelblue', density=True)
ax.axvline(0, color='red', linestyle='--', lw=2, label='H₀: разница = 0')
ax.axvline(np.mean(test) - np.mean(control), color='green', linestyle='-', lw=2, 
           label=f'Наблюдаемая разница = {np.mean(test) - np.mean(control):.1f}')
ax.axvline(ci[0], color='orange', linestyle=':', lw=2)
ax.axvline(ci[1], color='orange', linestyle=':', lw=2, label=f'95% ДИ')

ax.set_xlabel('Разница средних (Test - Control)', fontsize=12)
ax.set_ylabel('Плотность', fontsize=12)
ax.set_title('Bootstrap распределение разницы средних', fontsize=14)
ax.legend()
plt.tight_layout()
plt.show()

---

# 8. ? Методы повышения чувствительности

## Зачем повышать чувствительность?

Чем выше чувствительность (мощность) теста:
- ✅ Меньше пользователей нужно для эксперимента
- ✅ Быстрее получаем результат
- ✅ Детектируем меньшие эффекты

### Основные методы:
1. **Стратификация** — учёт структуры данных
2. **CUPED** — использование предэкспериментальных данных
3. **Работа с выбросами** — winsorization, trimming

## 8.1 Стратификация

<img src="pictures/strats.png" width="700">

### Что такое стратификация?

> **Стратификация** — разбиение пользователей на группы (страты) по признакам до эксперимента.

### Примеры страт:
- 📱 IOS vs Android
- 🏙️ Москва vs Регионы  
- 👤 Новички vs Старые пользователи

### Почему это работает?
- Убираем **межгрупповую дисперсию**
- Оставляем только **внутригрупповую**
- Результат: меньше шума → больше мощность

In [ ]:
# Демонстрация эффекта стратификации
def stratified_sampling(strata_params, strata_props, n_total, effect=0):
    """Стратифицированное сэмплирование"""
    np.random.seed(None)
    dfs_control, dfs_test = [], []
    
    for stratum, prop in strata_props.items():
        n_stratum = int(n_total * prop)
        mu = strata_params[stratum]['mu']
        sigma = strata_params[stratum]['sigma']
        
        vals_c = np.random.normal(loc=mu, scale=sigma, size=n_stratum)
        vals_t = np.random.normal(loc=mu, scale=sigma, size=n_stratum) * (1 + effect)
        
        dfs_control.append(pd.DataFrame({'stratum': stratum, 'value': vals_c}))
        dfs_test.append(pd.DataFrame({'stratum': stratum, 'value': vals_t}))
    
    return pd.concat(dfs_control), pd.concat(dfs_test)

def random_sampling(strata_params, strata_props, n_total, effect=0):
    """Случайное сэмплирование (без стратификации)"""
    strata = list(strata_props.keys())
    props = list(strata_props.values())
    
    chosen_control = np.random.choice(strata, size=n_total, p=props)
    chosen_test = np.random.choice(strata, size=n_total, p=props)
    
    vals_c = [np.random.normal(strata_params[s]['mu'], strata_params[s]['sigma']) 
              for s in chosen_control]
    vals_t = [np.random.normal(strata_params[s]['mu'], strata_params[s]['sigma']) * (1 + effect)
              for s in chosen_test]
    
    return pd.DataFrame({'stratum': chosen_control, 'value': vals_c}), \
           pd.DataFrame({'stratum': chosen_test, 'value': vals_t})

# Параметры страт (Москва vs Регионы)
strata_params = {
    'Москва': {'mu': 500, 'sigma': 150},
    'Регионы': {'mu': 350, 'sigma': 70},
}
strata_props = {'Москва': 0.7, 'Регионы': 0.3}

In [ ]:
# Сравнение дисперсии: случайное vs стратифицированное сэмплирование
np.random.seed(42)
n_simulations = 5000
n_total = 500

delta_random = []
delta_stratified = []

for _ in range(n_simulations):
    # Случайное сэмплирование
    ctrl_rand, test_rand = random_sampling(strata_params, strata_props, n_total)
    delta_random.append(test_rand['value'].mean() - ctrl_rand['value'].mean())
    
    # Стратифицированное
    ctrl_strat, test_strat = stratified_sampling(strata_params, strata_props, n_total)
    delta_stratified.append(test_strat['value'].mean() - ctrl_strat['value'].mean())

# Визуализация
fig, ax = plt.subplots(figsize=(10, 5))
sns.kdeplot(delta_random, fill=True, color='royalblue', label=f'Случайное (σ²={np.var(delta_random):.1f})', alpha=0.6)
sns.kdeplot(delta_stratified, fill=True, color='orange', label=f'Стратифицированное (σ²={np.var(delta_stratified):.1f})', alpha=0.6)

ax.axvline(0, color='red', linestyle='--', lw=2)
ax.set_xlabel('Дельта средних', fontsize=12)
ax.set_ylabel('Плотность', fontsize=12)
ax.set_title('Распределение дельт: случайное vs стратифицированное сэмплирование', fontsize=14)
ax.legend()
plt.tight_layout()
plt.show()

print(f"Снижение дисперсии: {100*(1 - np.var(delta_stratified)/np.var(delta_random)):.1f}%")

## 8.2 CUPED — Controlled-experiment Using Pre-Experiment Data

<img src="pictures/CUPED.png" width="600">

### Идея CUPED

> Используем **предэкспериментальные данные** для снижения дисперсии.

### Формула CUPED:

$$\hat{Y}^{cuped} = Y - \theta \cdot (X - \bar{X})$$

Где:
- $Y$ — метрика во время эксперимента
- $X$ — та же метрика **до** эксперимента  
- $\theta = \dfrac{Cov(X, Y)}{Var(X)}$ — оптимальный коэффициент

### Снижение дисперсии:

$$Var(\hat{Y}^{cuped}) = Var(Y) \cdot (1 - \rho^2_{XY})$$

Где $\rho_{XY}$ — корреляция между пред- и пост-экспериментальными данными.

In [ ]:
# Демонстрация CUPED
def cuped_adjustment(y_test, y_ctrl, x_test, x_ctrl):
    """
    Применяет CUPED корректировку.
    y - метрика во время эксперимента
    x - метрика до эксперимента (ковариата)
    """
    # Объединяем для расчёта theta
    y_all = np.concatenate([y_test, y_ctrl])
    x_all = np.concatenate([x_test, x_ctrl])
    
    # Оптимальный коэффициент
    theta = np.cov(x_all, y_all)[0, 1] / np.var(x_all)
    
    # CUPED корректировка
    x_mean = np.mean(x_all)
    y_test_cuped = y_test - theta * (x_test - x_mean)
    y_ctrl_cuped = y_ctrl - theta * (x_ctrl - x_mean)
    
    return y_test_cuped, y_ctrl_cuped, theta

# Симуляция: генерируем коррелированные пред- и пост-данные
np.random.seed(42)
n = 1000
correlation = 0.7  # Корреляция между пред- и пост-данными

# Генерация коррелированных данных
mean = [500, 500]
cov = [[100**2, correlation * 100**2], 
       [correlation * 100**2, 100**2]]

# Контроль
pre_ctrl, post_ctrl = np.random.multivariate_normal(mean, cov, n).T

# Тест (добавляем эффект 3%)
effect = 0.03
pre_test, post_test_base = np.random.multivariate_normal(mean, cov, n).T
post_test = post_test_base * (1 + effect)

# Без CUPED
delta_no_cuped = np.mean(post_test) - np.mean(post_ctrl)
se_no_cuped = np.sqrt(np.var(post_test)/n + np.var(post_ctrl)/n)

# С CUPED
post_test_cuped, post_ctrl_cuped, theta = cuped_adjustment(post_test, post_ctrl, pre_test, pre_ctrl)
delta_cuped = np.mean(post_test_cuped) - np.mean(post_ctrl_cuped)
se_cuped = np.sqrt(np.var(post_test_cuped)/n + np.var(post_ctrl_cuped)/n)

print(f"Корреляция пред/пост: {correlation}")
print(f"\n--- Без CUPED ---")
print(f"Дельта: {delta_no_cuped:.2f}, SE: {se_no_cuped:.2f}")
print(f"t-статистика: {delta_no_cuped/se_no_cuped:.2f}")

print(f"\n--- С CUPED ---")
print(f"Дельта: {delta_cuped:.2f}, SE: {se_cuped:.2f}")
print(f"t-статистика: {delta_cuped/se_cuped:.2f}")

print(f"\n🎯 Снижение SE: {100*(1 - se_cuped/se_no_cuped):.1f}%")
print(f"📈 Теоретическое снижение: {100*(1 - np.sqrt(1 - correlation**2)):.1f}%")

---

# 9. 👁️ Проблема подглядывания (Peeking Problem)

## Что такое подглядывание?

> **Подглядывание** — многократная проверка результатов эксперимента до его завершения.

### Почему это опасно?

Если смотреть на p-value каждый день и останавливаться когда p < 0.05:
- **Ошибка I рода сильно возрастает!**
- Вместо 5% можем получить 20-30% ложных срабатываний

In [ ]:
# Визуализация проблемы подглядывания
np.random.seed(42)
n_experiments = 15
total_n = 500
steps = np.arange(20, total_n + 1, 10)

pval_matrix = []

for _ in range(n_experiments):
    # AA-тест (эффекта нет!)
    control = np.random.normal(0, 1, total_n)
    test = np.random.normal(0, 1, total_n)
    
    pvals = []
    for step in steps:
        _, p = stats.ttest_ind(test[:step], control[:step], equal_var=False)
        pvals.append(p)
    pval_matrix.append(pvals)

pval_matrix = np.array(pval_matrix)

# Какие эксперименты хотя бы раз показали p < 0.05?
ever_significant = (pval_matrix < 0.05).any(axis=1)

# Визуализация
fig, ax = plt.subplots(figsize=(12, 6))

for idx in range(n_experiments):
    if ever_significant[idx]:
        ax.plot(steps, pval_matrix[idx], color='crimson', lw=2, alpha=0.7,
               label='Ложное срабатывание' if idx == np.where(ever_significant)[0][0] else "")
    else:
        ax.plot(steps, pval_matrix[idx], color='navy', lw=2, alpha=0.5,
               label='Нет срабатывания' if idx == np.where(~ever_significant)[0][0] else "")

ax.axhline(0.05, ls='--', color='green', lw=2, label='Порог α = 0.05')
ax.set_xlabel('Размер выборки в каждой группе', fontsize=12)
ax.set_ylabel('p-value', fontsize=12)
ax.set_title('Проблема подглядывания: AA-тесты без реального эффекта', fontsize=14)
ax.legend()
ax.set_ylim(0, 1)
plt.tight_layout()
plt.show()

print(f"❌ Доля ложных срабатываний при подглядывании: {ever_significant.mean():.1%}")
print(f"✅ Ожидаемая ошибка I рода: 5%")

## Решения проблемы подглядывания

### 1. Фиксированный размер выборки
- Определить размер заранее по MDE
- Смотреть результат только в конце

### 2. Поправка Бонферрони
Если смотрим K раз, используем порог $\alpha/K$

### 3. Group Sequential Testing (GST)
- **O'Brien-Fleming** — строгие пороги в начале, мягче к концу
- **Pocock** — одинаковый порог на всех этапах

In [ ]:
# Сравнение методов коррекции подглядывания
def obrien_fleming_bounds(K, alpha=0.05):
    """O'Brien-Fleming границы для K промежуточных анализов"""
    bounds = []
    for k in range(1, K + 1):
        prob = 1 - alpha / (2 * np.sqrt(K / k))
        bounds.append(stats.norm.ppf(prob))
    return np.array(bounds)

def bonferroni_bounds(K, alpha=0.05):
    """Поправка Бонферрони"""
    z = stats.norm.ppf(1 - alpha / (2 * K))
    return np.array([z] * K)

K = 5  # Число промежуточных анализов

obf = obrien_fleming_bounds(K)
bonf = bonferroni_bounds(K)
standard = np.array([1.96] * K)  # Без коррекции

# Визуализация
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(range(1, K+1), standard, 'g--o', lw=2, label='Без коррекции (z=1.96)', markersize=8)
ax.plot(range(1, K+1), bonf, 'b-o', lw=2, label='Бонферрони', markersize=8)
ax.plot(range(1, K+1), obf, 'r-o', lw=2, label="O'Brien-Fleming", markersize=8)

ax.fill_between(range(1, K+1), -standard, standard, alpha=0.1, color='green')

ax.set_xlabel('Номер промежуточного анализа', fontsize=12)
ax.set_ylabel('Критическое значение z-статистики', fontsize=12)
ax.set_title('Методы коррекции множественных проверок', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xticks(range(1, K+1))
plt.tight_layout()
plt.show()

print(f"При {K} промежуточных анализах:")
print(f"  Без коррекции: z = 1.96 (α раздувается!)")
print(f"  Бонферрони:    z = {bonf[0]:.2f}")
print(f"  O'Brien-Fleming: от {obf[0]:.2f} до {obf[-1]:.2f}")

---

# 10. 🎯 Множественное тестирование

## Проблема множественных сравнений

Когда тестируем **много гипотез** одновременно:
- Много метрик в одном эксперименте
- Много сегментов пользователей
- Много вариантов теста (A/B/C/D...)

### Пример:
Тестируем 20 метрик с α = 0.05

**Вероятность хотя бы одного ложного срабатывания:**
$$P(\text{хотя бы 1 FP}) = 1 - (1 - 0.05)^{20} = 64\%$$

In [ ]:
# Демонстрация проблемы множественного тестирования
from statsmodels.stats.multitest import multipletests

np.random.seed(42)
n_metrics = 20
n_simulations = 1000

# Симуляция: все метрики без эффекта (H0 верна для всех)
false_positives_none = 0  # Без коррекции
false_positives_bonf = 0  # С Бонферрони
false_positives_bh = 0    # С Benjamini-Hochberg

for _ in range(n_simulations):
    pvalues = []
    for _ in range(n_metrics):
        control = np.random.normal(0, 1, 500)
        test = np.random.normal(0, 1, 500)  # Нет эффекта!
        _, p = stats.ttest_ind(test, control)
        pvalues.append(p)
    
    pvalues = np.array(pvalues)
    
    # Без коррекции
    if (pvalues < 0.05).any():
        false_positives_none += 1
    
    # Бонферрони
    _, pvals_bonf, _, _ = multipletests(pvalues, method='bonferroni')
    if (pvals_bonf < 0.05).any():
        false_positives_bonf += 1
    
    # Benjamini-Hochberg (FDR)
    _, pvals_bh, _, _ = multipletests(pvalues, method='fdr_bh')
    if (pvals_bh < 0.05).any():
        false_positives_bh += 1

print(f"Тестируем {n_metrics} метрик без эффекта:")
print(f"\n{'Метод':<25} {'Ложные срабатывания':>20}")
print("-" * 45)
print(f"{'Без коррекции':<25} {false_positives_none/n_simulations:>19.1%}")
print(f"{'Бонферрони (FWER)':<25} {false_positives_bonf/n_simulations:>19.1%}")
print(f"{'Benjamini-Hochberg (FDR)':<25} {false_positives_bh/n_simulations:>19.1%}")

## Методы коррекции

### 1. Бонферрони (FWER контроль)
$$\alpha_{adj} = \dfrac{\alpha}{m}$$

- ✅ Строго контролирует вероятность любого ложного срабатывания
- ❌ Очень консервативен — теряем мощность

### 2. Benjamini-Hochberg (FDR контроль)
- Контролирует **долю** ложных срабатываний среди всех срабатываний
- ✅ Менее консервативен
- ✅ Хорошо подходит для разведочного анализа

### Практические рекомендации:
- **Главная метрика** — без коррекции (заранее выбрана)
- **Вспомогательные метрики** — коррекция FDR
- **Сегменты** — осторожно интерпретировать!

---

# 📚 Итоги: 10 ключевых тем A/B тестирования

| # | Тема | Главный вывод |
|---|------|---------------|
| 1 | **Введение в A/B** | A/B тест = рандомизированный контролируемый эксперимент |
| 2 | **Ошибки I и II рода** | α = P(FP), β = P(FN), нужен баланс |
| 3 | **MDE** | Заранее считаем размер выборки через желаемый эффект |
| 4 | **AA-тесты** | Валидация: p-values ~ Uniform(0,1) |
| 5 | **Агрегация данных** | Агрегируем по пользователям, не по событиям |
| 6 | **Дельта-метод** | Для ratio-метрик учитываем ковариацию |
| 7 | **Бутстреп** | Универсальный непараметрический метод |
| 8 | **CUPED + Стратификация** | Снижаем дисперсию → быстрее тесты |
| 9 | **Peeking problem** | GST методы для раннего останова |
| 10 | **Множественное тестирование** | Bonferroni (FWER) или BH (FDR) коррекция |

---

## 🎯 Главные практические советы

1. **До эксперимента**: выберите главную метрику, посчитайте MDE и размер выборки
2. **Рандомизация**: сплит по user_id, проверьте AA-тестом
3. **Метрики**: агрегируйте на уровне пользователей, используйте CUPED для ускорения
4. **Анализ**: не подглядывайте! Используйте GST если нужна ранняя остановка
5. **Множественность**: коррекция для вспомогательных метрик и сегментов

# 🙋 Вопросы?

<div style="text-align: center; padding: 50px;">
    <h2>Спасибо за внимание!</h2>
    <p>Материалы: <code>AB Week. Day 1-5.ipynb</code></p>
</div>